# Spine Segmentation

In [ ]:
# import libs

import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from random import randint
import cv2
import os
import pydicom
import json

from func import search_for_borders, get_edgefunc_coef, get_vertebras_corners, get_ratio, approximateYbezie_lcr, adjast_vertebras, MSE

import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence

In [ ]:
# Init global variables

DATA_DIR = os.path.abspath('../Data/spine-segmentation')
with open(os.path.join(DATA_DIR, 'files_routes.json'), 'r') as f:
    FILE_DIRS = json.loads(f.read())

RESULTS = os.path.abspath('../Results')

IMG_SHAPE = (3408, 1552)
VALIDATION_NUM = 1
BATCH_SIZE = 3

WEIGHTS2LOAD = os.path.join(RESULTS, 'saved_weights/10-weights_unetpp.hdf5')

In [ ]:
# Data Generator

class Data_train_generator(Sequence):
    def __init__(self, x_files_list: list, y_files_list: list, batch_size: int, new_image_size = None, shuffle = True) -> None:
        self.data = x_files_list
        self.labels = y_files_list
        self.batch_size = batch_size

        self.shuffle = shuffle
        self.new_image_size = new_image_size

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __open_png_y(self, file_path):
        img = cv2.imread(file_path)
        img = cv2.inRange(img, (20, 20, 210), (40, 40, 240))
        
        data = np.array(img, dtype="float32") / 255

        if self.new_image_size != None:
            data = cv2.resize(data, self.new_image_size[::-1], interpolation=cv2.INTER_CUBIC)

        return np.reshape(data, (*self.new_image_size, 1))

    def __open_dcm_x(self, file_path):
        dcm = pydicom.dcmread(file_path)

        data = dcm.pixel_array.astype("float32") / 255

        if self.new_image_size != None:
            data = cv2.resize(data, self.new_image_size[::-1], interpolation=cv2.INTER_CUBIC)

        return np.reshape(data, (*self.new_image_size, 1))

    def __getitem__(self, index):
        batch_x = np.array(list(map(self.__open_dcm_x, self.data[index * self.batch_size: (index + 1) * self.batch_size])))
        batch_y = np.array(list(map(self.__open_png_y, self.labels[index * self.batch_size: (index + 1) * self.batch_size])))

        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            for i in range(len(self.data)):
                ind_from, ind_to = randint(0, len(self.data) - 1), randint(0, len(self.data) - 1)
                self.data[ind_from], self.data[ind_to] = self.data[ind_to], self.data[ind_from]
                self.labels[ind_from], self.labels[ind_to] = self.labels[ind_to], self.labels[ind_from]

In [ ]:
# Model Init

from model import make_unet2p

model_unet = make_unet2p((*IMG_SHAPE, 1), filters=[64, 128, 256, 512, 1024], deep_supervision=True)

if WEIGHTS2LOAD: model_unet.load_weights(WEIGHTS2LOAD)

In [ ]:
# Testing

dcm = pydicom.dcmread(os.path.join(DATA_DIR, 'IMG-0001-00002.dcm'))
data = dcm.pixel_array.astype("float32") / 255

if IMG_SHAPE != None:
    data = cv2.resize(data, IMG_SHAPE[::-1], interpolation=cv2.INTER_CUBIC)

data = np.reshape(data, (1, *IMG_SHAPE, 1))

res = model_unet.predict(data, batch_size=1)[3].reshape(IMG_SHAPE)

# Drawing Result

fig, ax = plt.subplots(nrows=1, ncols=2)

sns.heatmap(res, ax=ax[0])
sns.heatmap(np.where(res > 0.5, 1, 0), ax=ax[1])

plt.show()

In [ ]:
# Training To Point Vertebras

IMG_PATH = os.path.join(DATA_DIR, 'test/IMG-0001-00001.png')

img = cv2.inRange(cv2.imread(IMG_PATH), (20, 20, 210), (40, 40, 240))

data = np.array(img, dtype="float32") / 255

n = 10
threshold = 0.7

border_coords = search_for_borders(data)

edge_coef = get_edgefunc_coef(data, border_coords, n)

y, y_l, y_r = approximateYbezie_lcr(border_coords, edge_coef, n)

vertebras_corners = get_vertebras_corners(data, border_coords, y, y_l, y_r, threshold)

#vertebras_corners = adjast_vertebras(border_coords, vertebras_corners, y, y_l, y_r)
#vertebras_corners_true = adjast_vertebras(border_coords_true, vertebras_corners_true, y_true, y_l_true, y_r_true) 
# It can't be used due to a huge size of S1 part

#error = MSE(vertebras_corners, vertebras_corners_true)

# Point Vertebras

vertebras_corners = [[[vertebras_corners[i], vertebras_corners[i + 1]], [vertebras_corners[i + 2], vertebras_corners[i + 3]]] for i in range(0, len(vertebras_corners), 4)]

In [ ]:
# plotting Info about training

with open(os.path.join(RESULTS, 'model_history_1'), 'rb') as f:
    hist = pickle.load(f)

fig, ax = plt.subplots(nrows=2, ncols=2)

sns.lineplot(hist["loss"], ax=ax[0][0])
sns.lineplot(hist["val_loss"], ax=ax[0][1])

sns.lineplot(hist["output_4_loss"], ax=ax[1][0])
sns.lineplot(hist["val_output_4_loss"], ax=ax[1][1])